# Chapter 06. 학습 관련 기술들

신경망 학습의 핵심 개념

- 가중치 매개변수의 최적값을 탐색하는 방법
- 가중치 매개변수의 초깃값
- Hyperparameter 설정 방법
- 오버피팅 대응책: 가중치 감소와 드롭아웃의 정규화 방법

$\Rightarrow$ 신경망(딥러닝)학습의 효율, 정확도 향상


## 6.1 매개변수 갱신

신경망 학습의 목적: 손실함수의 값을 최소화하는 가중치 매개변수 찾기 (매개변수의 최적값을 찾는 문제를 푸는 것: **최적화(optimization)**)

앞장에서는 SGD(Stochastic Gradient Descent, 확률적 경사 하강법)를 사용했다.

### 6.1.2 확률적 경사 하강법(SGD)

$W \leftarrow W - \eta \frac{\partial L}{\partial W}$

- $W$: 갱신할 가중치 매개변수
- $\frac{\partial L}{\partial W}$: 손실함수의 기울기
- $\eta$: learning rate

In [1]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

In [2]:
# Pesudo code
network = TwoLayerNet(...)
optimizer = SGD()

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...)  # mini-batch
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ...

NameError: name 'TwoLayerNet' is not defined

### 6.1.3 SGD의 단점

$f(x,y) = \frac{1}{20}x^2 + y^2$ ($x$축 방향으로 늘인 타원 모양)
- 이 함수의 기울기는 $y$축 방향은 크고, $x$축 방향은 작다. ($y$축 방향은 가파르고 $x$축 방향은 완만하다.)
- 최소값을 갖는 장소 $(x,y) = (0,0)$, 기울기의 대부분은 (0,0)을 가리키지 않는다.

**SGD의 단점**: **비등방성(anisotripy) 함수** (방향에 따라 성질, 즉 여기에서는 기울기가 달라지는 함수)에서는 탐색 경로가 비효율적이라는 것.

이런한 SGD의 단점을 개선해주는 **모멘텀, AdaGrad, Adam** 방법이 있다.


### 6.1.4 모멘텀

**모멘텀(Momentum)**: '운동량'을 뜻하는 단어, 물리와 관계 있다.

$v \leftarrow \alpha v - \eta \frac{\partial L}{\partial W}$

$W \leftarrow W + v$

- $W$: 가중치 매개변수
- $\frac{\partial L}{\partial W}$: $W$에 대한 손실 함수의 기울기
- $\eta$ : 학습률
- $v$: 속도(velocity)
- $\alpha v$: 물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할 ($\alpha: 0.9등의 값으로 설정)

In [3]:
# 모멘텀 구현

class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

인스턴스 변수 v가 물체의 속도. v는 초기화 때는 아무 값도 담지 않고, 대신 update()가 처음 호출될 때 매개변수와 같은 구조의 데이터를 딕셔너리 변수로 저장한다. 모멘텀의 갱신 경로는 공의 그릇 바닥을 구르듯 움직인다. SGD와 비교하면 '지그재그'가 덜하다. $x$축의 힘은 아주 작지만 방향은 변하지 않아서 한 방향으로 일정하게 가속하기 때문이다. $y$축의 힘은 크지만 위아래로 번갈아 받아서 상충하여 $y$축 방향의 속도는 안정적이지 않다. 

### 6.1.5 AdaGrad

신경망에서 학습률($\eta$)값은 중요. 이 값이 너무 작으면 학습 시간이 너무 길어지고, 너무 크면 발산하여 학습이 제대로 이뤄지지 않는다.

학습률을 정하는 효과적 기술로 **학습률 감소(learning rate decay)** 가 있다. 학습을 진행하면서 학습률을 점차 줄여가는 방법이 있다. 처음에는 크게 학습하다가 조금씩 작게 학습한다는 것.

학습률 서서히 낮추는 가장 간단한 방법: 매개변수 **전체**의 학습률 값을 일괄적으로 낮춰 $\rightarrow$ **AdaGrads**: **각각**의 매개변수 **맞춤형** 값 만든다.

AdaGrads는 개별 매개변수에 **적응형(adaptive)** 로 학습률 조정하며 학습 진행

$h \leftarrow h + \frac{\partial L}{\partial W} \odot \frac{\partial L}{\partial W}$ ($\odot$: 행렬의 원소별 곱)

$W \leftarrow W - \eta \frac{1}{\sqrt{h}} \frac{\partial L}{\partial W}$

- W: 갱신할 가중치 매개변수
- $\eta$: 학습률
- $\frac{\partial L}{\partial W}$: W에 대한 손실함수의 기울기
- $h$: 기존 기울기 값을 제곱 후 계속 더해
- $\frac{1}{\sqrt{h}}$을 곱해 학습률 조정

매개변수 원소 중 많이 움직인(크게 갱신된) 원소는 학습률 낮아짐 $\rightarrow$ 학습률 감소가 매개변수의 원소마다 다르게 적용

NOTE) AdaGrads는 과거의 기울기 제곱하고 덧셈, 학습 진행할수록 갱신 강도 낮아지고, 무한히 학습 시 갱신량 0이 된다.

In [1]:
# AdaGrads 구현

class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

y축 방향은 기울기가 커서 처음에는 크게 움직임, 큰 움직임에 비례해 갱신 정도도 큰 폭으로 작아지도록 조정됨. 

y축 방향으로 갱신 강도가 빠르게 약해지고, 지그재그 움직임이 줄어든다.

### 6.1.6 Adam
Adam = Momentum + AdaGrad

